In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('GH_Delivery.csv')
df = df.iloc[:,1:]
df

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120.0,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,NaN,19:50:00,Stormy,Jam,scooter,Metropolitian,165.0,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130.0,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,NaN,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150.0,Toys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43734,jlxf819993117,30,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,11:35:00,11:45:00,Windy,High,motorcycle,Metropolitian,160.0,Home
43735,aevx342135787,21,4.6,infinite,0.000000,0.070000,0.070000,2022-02-16,19:55:00,20:10:00,Windy,Jam,motorcycle,Metropolitian,180.0,Jewelry
43736,xnek760674819,30,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,23:50:00,00:05:00,Cloudy,Low,scooter,Metropolitian,80.0,Home
43737,cynl434665991,20,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,13:35:00,13:40:00,Cloudy,High,motorcycle,Metropolitian,130.0,Kitchen


In [3]:
df[df.isna().any(axis=1)]

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,NaN,Cosmetics
26,qvfd540651807,35,4.3,21.160522,72.771477,21.250522,72.861477,2022-03-12,23:20:00,23:30:00,Cloudy,Low,scooter,Metropolitian,NaN,Electronics
52,fris321432357,28,4.8,11.003681,76.975525,11.083681,77.055525,2022-04-04,NaN,20:55:00,Sandstorms,Jam,motorcycle,Metropolitian,NaN,Kitchen
124,uurs547552548,23,NaN,22.569358,88.433452,22.599358,88.463452,2022-02-17,23:25:00,23:35:00,Sunny,Low,motorcycle,Metropolitian,75.0,Snacks
153,dsbe906886027,29,4.3,22.538731,88.364878,22.598731,88.424878,2022-02-13,23:50:00,23:55:00,Sandstorms,Low,motorcycle,Metropolitian,NaN,Cosmetics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43195,mhmv509867634,21,4.9,17.41233,78.449654,17.442330,78.479654,2022-03-21,22:50:00,23:05:00,Fog,Low,motorcycle,Metropolitian,NaN,Kitchen
43198,zqpp684328510,20,4.8,12.914264,77.678400,12.974264,77.738400,2022-04-01,20:30:00,20:45:00,Cloudy,Jam,motorcycle,Metropolitian,NaN,Jewelry
43427,klfo304458546,23,4.4,12.334022,76.618203,12.414022,76.698203,2022-03-08,20:20:00,20:30:00,Stormy,Jam,motorcycle,Metropolitian,NaN,Electronics
43455,icby656235480,22,4.3,22.307898,73.167788,22.397898,73.257788,2022-03-31,18:25:00,18:35:00,Fog,Medium,scooter,Metropolitian,NaN,Jewelry


In [4]:
df.isna().value_counts()

Order_ID  Agent_Age  Order_Rating  Store_Latitude  Store_Longitude  Drop_Latitude  Drop_Longitude  Order_Date  Order_Time  Pickup_Time  Weather  Traffic  Vehicle  Area   Delivery_Time  Category
False     False      False         False           False            False          False           False       False       False        False    False    False    False  False          False       43161
                                                                                                                                                                          True           False         435
                                                                                                                                        True     False    False    False  False          False          90
                     True          False           False            False          False           False       False       False        False    False    False    False  False          False       

In [10]:
#df = df[df.notna().all(axis=1) | df['Order_Time'].apply(lambda x: np.nan if x == 'NaN' else x).notna()]
df['Agent_Age'].value_counts()

35    2064
36    2032
30    2001
37    1997
24    1995
22    1986
29    1977
38    1975
33    1971
28    1969
32    1967
34    1950
26    1948
25    1942
27    1940
39    1940
20    1932
21    1920
23    1907
31    1898
15     879
50      44
Name: Agent_Age, dtype: int64

In [6]:
pd.options.display.min_rows = 40

df = df[df['Store_Latitude'] !='infinite']
df

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120.0,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,NaN,19:50:00,Stormy,Jam,scooter,Metropolitian,165.0,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130.0,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,NaN,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150.0,Toys
5,fxuu788413734,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,Urban,130.0,Toys
6,njmo150975311,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,19:30:00,Fog,Jam,scooter,Metropolitian,200.0,Toys
7,jvjc772545076,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,17:30:00,Cloudy,Medium,motorcycle,Metropolitian,160.0,Snacks
8,uaeb808891380,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,21:05:00,Stormy,Jam,motorcycle,Metropolitian,170.0,Electronics
9,bgvc052754213,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,22:10:00,Fog,Jam,motorcycle,Metropolitian,230.0,Toys
